# Autoencoders

> Collection of Autoencoder models

In [ ]:
#| default_exp models.autoencoders

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
import torch.nn.functional as F
import torch.nn as nn
import torch
from nimrod.modules import Encoder, Decoder
from lightning import LightningModule
from nimrod.image.datasets import MNISTDataset
from torch.utils.data import DataLoader

In [ ]:
#| export
class AutoEncoder(nn.Module):
    def __init__(self,
        encoder:Encoder, # Encoder layer
        decoder:Decoder # Decoder layer
        ):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self,
        x:torch.Tensor # Tensor B x L
        )->torch.Tensor:
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [ ]:
enc = Encoder()
dec = Decoder()
a = AutoEncoder(enc, dec)
batch = torch.rand((10, 28*28))
y = a(batch)
print(y.shape)

In [ ]:
ds = MNISTDataset(data_dir='../data/image/')
dl = DataLoader(ds)
b = next(iter(dl))
print(len(b), b[0].shape, b[1].shape)


In [ ]:
#| export
class AutoEncoderPL(LightningModule):
    def __init__(self, autoencoder:AutoEncoder):
        super().__init__()
        # self.save_hyperparameters()
        self.save_hyperparameters(ignore=['autoencoder'])
        self.autoencoder = autoencoder
        self.metric = torch.nn.MSELoss()

    def forward(self, x):
        return self.autoencoder(x)
    
    def predict_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        with torch.no_grad():
            return self.autoencoder(x)

    def _shared_eval(self, batch, batch_idx, prefix, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        x, _ = batch
        x = x.view(x.size(0), -1) # flatten B x C x H x W to B x L (grey pic)
        x_hat = self.autoencoder(x)
        loss = self.metric(x_hat, x)
        self.log(f"{prefix}/loss", loss, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)
        return loss

    def training_step(self, batch, batch_idx):
        return self._shared_eval(batch, batch_idx, "train")
    
    def test_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "test")
    
    def validation_step(self, batch, batch_idx):
        return self._shared_eval(batch, batch_idx, "val")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [ ]:
#| hide
# def on_validation_batch_end(
#         self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx, wadn
#         ):
#         # `outputs` comes from `LightningModule.validation_step`
#         # which corresponds to our model predictions in this case
        
#         # Let's log 20 sample image predictions from the first batch
#         if batch_idx == 0:
#             n = 20
#             x, y = batch
#             images = [img for img in x[:n]]
#             captions = [f'Ground Truth: {y_i} - Prediction: {y_pred}' 
#                 for y_i, y_pred in zip(y[:n], outputs[:n])]
            
            
#             # Option 1: log images with `WandbLogger.log_image`
#             wandb_logger.log_image(
#                 key='sample_images', 
#                 images=images, 
#                 caption=captions)


In [ ]:
autoencoder_pl = AutoEncoderPL(a)
b = torch.rand((5,28*28))
y = autoencoder_pl(b)
print(y.shape)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()